In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import os

In [ ]:
mp_pose = mp.solutions.pose.Pose()

# 랜드마크를 추출할 폴더 경로
# -------------------------------------
folder_path = "dataset"
# -------------------------------------

file_list = os.listdir(folder_path)

In [ ]:
# for index in range(len(file_list)):
for index, video in enumerate(file_list):
    # 폴더 내 각 동영상의 경로
    # -------------------------------------
    video_path = folder_path + "/" + video
    # -------------------------------------

    # 랜드마크 데이터셋 저장 디렉토리 설정
    # -------------------------------------
    output_dir = "landmark/" + str(index)
    # -------------------------------------

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 동영상 열기
    cap = cv2.VideoCapture(video_path)

    frame_number = 0

    while cap.isOpened():
        ret, image = cap.read()
        if not ret:
            break

        # BGR 이미지를 RGB 이미지로 변환 (Mediapipe는 RGB 이미지를 입력으로 받음)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # 포즈 추정 수행
        results = mp_pose.process(image_rgb)

        if results.pose_landmarks:
            # 각 랜드마크의 좌표를 소수점 4자리까지 추출하여 데이터셋에 저장
            data = []
            for landmark_id, landmark in enumerate(results.pose_landmarks.landmark):
                h, w, c = image.shape
                x, y, z, visibility = (
                    round(float(landmark.x), 4),
                    round(float(landmark.y), 4),
                    round(float(landmark.z), 4),
                    round(float(landmark.visibility), 4),
                )

                data.append(x)
                data.append(y)
                data.append(z)
                data.append(visibility)

            # 데이터셋을 (33,4) 크기의 numpy 배열로 변환하여 저장
            data = np.array(data).reshape(33, 4)
            # 프레임 번호를 파일 이름으로 지정하여 저장
            file_name = os.path.join(output_dir, f"{frame_number}.npy")
            np.save(file_name, data)

            # 이미지에 랜드마크를 시각화
            mp.solutions.drawing_utils.draw_landmarks(
                image, results.pose_landmarks, mp.solutions.pose.POSE_CONNECTIONS
            )

        # 결과를 화면에 출력
        cv2.imshow("Pose Estimation", image)
        if cv2.waitKey(1) & 0xFF == 27:
            break

        frame_number += 1

    cap.release()
    cv2.destroyAllWindows()